In [9]:
import sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols
import MathsUtilities as MUte
%matplotlib inline

In [10]:
Colors = ['darkblue',           
'darkcyan',           
'darkgoldenrod',      
'darkgray',           
'darkgreen',          
'darkkhaki',          
'darkmagenta',        
'darkolivegreen',     
'darkorange',         
'darkorchid',         
'darkred',            
'darkviolet',
'blue',           
'cyan',           
'goldenrod',      
'gray',           
'green',
'black']
Markers = ['o','s','^']

In [11]:
con = sqlite3.connect('C:\GitHubRepos\ApsimX\Prototypes\Lucerne/LucerneValidation.db')

In [12]:
PredictedObserved = pd.read_sql("Select * from PredictedObserved",
                        con)
#PredictedObserved.set_index(['SimulationID','Clock.Today'],inplace=True)

In [13]:
Report = pd.read_sql("Select * from Report", con)
Report.set_index(['SimulationID','Clock.Today'],inplace=True)

In [14]:
PredictedObserved.columns

Index(['CheckpointID', 'Observed.CheckpointID', 'Predicted.CheckpointID',
       'SimulationID', 'Clock.Today', 'Observed.SWC', 'Predicted.SWC',
       'Observed.StemWt', 'Predicted.StemWt', 'Pred-Obs.StemWt',
       'Observed.LeafWt', 'Predicted.LeafWt', 'Pred-Obs.LeafWt',
       'Observed.RootWt', 'Predicted.RootWt', 'Pred-Obs.RootWt', 'Observed.ET',
       'Predicted.ET', 'Pred-Obs.ET', 'LAI', 'Observed.Height',
       'Predicted.Height', 'Pred-Obs.Height'],
      dtype='object')

In [15]:
Factors = pd.read_sql("Select * from _Factors",
                        con)
ValidationIndex = Factors.drop_duplicates(subset=['SimulationName']).loc[:,Factors.columns.isin(['SimulationID','ExperimentName','SimulatinName','FolderName'])]
Experiments = ValidationIndex.ExperimentName.drop_duplicates().tolist()
Folders = ValidationIndex.FolderName.drop_duplicates().tolist()

In [21]:
Report

CheckpointID Defoliation     Zone  \
SimulationID Clock.Today                                              
4            2000-11-14 12:00:00             1        None  paddock   
             2000-11-15 12:00:00             1        None  paddock   
             2000-11-16 12:00:00             1        None  paddock   
             2000-11-17 12:00:00             1        None  paddock   
             2000-11-18 12:00:00             1        None  paddock   
             2000-11-19 12:00:00             1        None  paddock   
             2000-11-20 12:00:00             1        None  paddock   
             2000-11-21 12:00:00             1        None  paddock   
             2000-11-22 12:00:00             1        None  paddock   
             2000-11-23 12:00:00             1        None  paddock   
             2000-11-24 12:00:00             1        None  paddock   
             2000-11-25 12:00:00             1        None  paddock   
             2000-11-26 12:00:00             1        None  paddock   
             2000-11-27 12:00:00             1        None  paddock   
             2000-11-28 12:00:00             1        None  paddock   
             2000-11-29 12:00:00             1        None  paddock   
             2000-11-30 12:00:00             1        None  paddock   
             2000-12-01 12:00:00             1        None  paddock   
             2000-12-02 12:00:00             1        None  paddock   
             2000-12-03 12:00:00             1        None  paddock   
             2000-12-04 12:00:00             1        None  paddock   
             2000-12-05 12:00:00             1        None  paddock   
             2000-12-06 12:00:00             1        None  paddock   
             2000-12-07 12:00:00             1        None  paddock   
             2000-12-08 12:00:00             1        None  paddock   
             2000-12-09 12:00:00             1        None  paddock   
             2000-12-10 12:00:00             1        None  paddock   
             2000-12-11 12:00:00             1        None  paddock   
             2000-12-12 12:00:00             1        None  paddock   
             2000-12-13 12:00:00             1        None  paddock   
...                                        ...         ...      ...   
57           1999-12-02 12:00:00             1     Cutting  paddock   
             1999-12-03 12:00:00             1     Cutting  paddock   
             1999-12-04 12:00:00             1     Cutting  paddock   
             1999-12-05 12:00:00             1     Cutting  paddock   
             1999-12-06 12:00:00             1     Cutting  paddock   
             1999-12-07 12:00:00             1     Cutting  paddock   
             1999-12-08 12:00:00             1     Cutting  paddock   
             1999-12-09 12:00:00             1     Cutting  paddock   
             1999-12-10 12:00:00             1     Cutting  paddock   
             1999-12-11 12:00:00             1     Cutting  paddock   
             1999-12-12 12:00:00             1     Cutting  paddock   
             1999-12-13 12:00:00             1     Cutting  paddock   
             1999-12-14 12:00:00             1     Cutting  paddock   
             1999-12-15 12:00:00             1     Cutting  paddock   
             1999-12-16 12:00:00             1     Cutting  paddock   
             1999-12-17 12:00:00             1     Cutting  paddock   
             1999-12-18 12:00:00             1     Cutting  paddock   
             1999-12-19 12:00:00             1     Cutting  paddock   
             1999-12-20 12:00:00             1     Cutting  paddock   
             1999-12-21 12:00:00             1     Cutting  paddock   
             1999-12-22 12:00:00             1     Cutting  paddock   
             1999-12-23 12:00:00             1     Cutting  paddock   
             1999-12-24 12:00:00             1     Cutting  paddock   
             1999-12-25 12:00:00             1     Cutting

In [17]:
def MakeLabel(RegStats):
    #text = RegStats.Name + '  n = ' + str(RegStats.n)
    text = '\ny = ' + '%.2f'%RegStats.Intercept + '(se ' + '%.2f'%RegStats.SEintercept + ') + ' + '%.2f'%RegStats.Slope + '(se ' + '%.2f'%RegStats.SEslope + ') x' 
    text += '\n$r^2$ =' + '%.2f'%RegStats.R2 + ' RMSE = ' + '%.2f'%RegStats.RMSE +' NSE = ' + '%.2f'%RegStats.NSE
    text += '\nME = ' + '%.2f'%RegStats.ME + ' MAE = ' + '%.2f'%RegStats.MAE
    text += '\nn = ' + str(RegStats.n)
    return text

In [18]:
Graph = plt.figure(figsize=(15,15))
Variables = ['Height','LAI','RootWt','StemWt','LeafWt']
Units = ['$g/m^3$','$/m^3$','$g/m^3$',
        '$g/m^3$','$g/m^3$','$g/m^3$',
        'days','days']
Names = ['Height','LAI','RootWt','StemWt','LeafWt']
Pos = 1
for Var in Variables:
    NaNFilter = np.isnan(PredictedObserved.loc[:,'Predicted.'+Var]) | np.isnan(PredictedObserved.loc[:,'Observed.'+Var])
    IaNFilter = ~NaNFilter
    Obs = PredictedObserved.loc[:,'Observed.'+Var].loc[IaNFilter].values
    Pred = PredictedObserved.loc[:,'Predicted.'+Var].loc[IaNFilter].values
    Graph.add_subplot(3,3,Pos)
    plt.plot(Obs,Pred,Markers[MarPos],color = Colors[ColPos],label=Fol)
    uplim = max(Obs.max(),Pred.max())*1.1
    lowlim = min(Obs.min(),Pred.min())*0.95
    plt.title(Names[Pos-1] + ' (' + Units[Pos-1] + ')')
    plt.ylim(0,uplim)
    plt.xlim(0,uplim)
    plt.plot([lowlim,uplim*.95],[lowlim,uplim*.95],'-')
    RegStats = MUte.MathUtilities.CalcRegressionStats(Var,Pred,Obs)
    LabelText = MakeLabel(RegStats)
    plt.text(uplim*0.05,uplim*.78,LabelText)
    #Fit linear regression to current series and store slope and intercept in dataframe
    ModFit = sm.regression.linear_model.OLS(Pred,  # Y variable
                                        sm.add_constant(Obs), # X variable
                                        missing='drop',                                     # ignor and data where one value is missing
                                        hasconst=False) 
    RegFit = ModFit.fit();  # fit models parameters
    Slope = RegFit.params[1] 
    Intercept = RegFit.params[0]
    Xs = [lowlim,uplim*.95]
    Ys = [Intercept + Xs[0]*Slope,Intercept + Xs[1]*Slope]
    plt.plot(Xs,Ys,'--')
    Pos +=1
plt.text(700,-500,'Observed',fontsize=20)
plt.text(-4400,3500,'Predicted',fontsize=20,rotation=90)
plt.legend(loc = (1.2,0),ncol=1,numpoints=1,markerscale=2)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

<Figure size 1080x1080 with 0 Axes>

In [ ]:
for pair in PredictedObserved.index:
    SimID =  PredictedObserved.loc[pair,'SimulationID']
    ObsDate = PredictedObserved.loc[pair,'Clock.Today']
    PredictedObserved.loc[pair,'DSWS'] = Report.loc[(SimID,ObsDate),'Weather.DaysSinceWinterSolstice']
    PredictedObserved.loc[pair,'DReg'] = Report.loc[(SimID,ObsDate),'Lucerne.Phenology.DaysAfterCutting']

In [ ]:
Graph = plt.figure(figsize=(15,15))
Variables = ['Height','LAI','RootWt','StemWt','LeafWt']
Units = ['$g/m^3$','$/m^3$','$g/m^3$',
        '$g/m^3$','$g/m^3$','$g/m^3$',
        'days','days']
Names = ['Height','LAI','RootWt','StemWt','LeafWt']
Pos = 1
for Var in Variables:
    NaNFilter = np.isnan(PredictedObserved.loc[:,'Predicted.'+Var]) | np.isnan(PredictedObserved.loc[:,'Observed.'+Var])
    IaNFilter = ~NaNFilter
    Obs = PredictedObserved.loc[:,'Observed.'+Var].loc[IaNFilter].values
    Pred = PredictedObserved.loc[:,'Predicted.'+Var].loc[IaNFilter].values
    residual = Pred - Obs
    DSWS = PredictedObserved.loc[:,'DSWS'].loc[IaNFilter].values
    ax = Graph.add_subplot(3,3,Pos)
    plt.plot(DSWS,residual,Markers[MarPos],color = Colors[ColPos],label=Fol)
    uplim = max(residual.max(),Pred.max())*1.1
    lowlim = min(residual.min(),Pred.min())*0.95
    plt.title(Names[Pos-1] + ' (' + Units[Pos-1] + ')')
    plt.ylim(lowlim,uplim)
    plt.xlim(0,365)
    RegStats = MUte.MathUtilities.CalcRegressionStats(Var,Pred,Obs)
    LabelText = MakeLabel(RegStats)
    plt.text(0.05,0.78,LabelText,transform=ax.transAxes)
    Pos +=1
plt.text(70,-50,'Days since winter solstice',fontsize=20)
plt.text(-40,30,'Residual',fontsize=20,rotation=90)
plt.legend(loc = (1.2,0),ncol=1,numpoints=1,markerscale=2)

In [ ]:
Graph = plt.figure(figsize=(15,15))
Variables = ['Height','LAI','RootWt','StemWt','LeafWt']
Units = ['$g/m^3$','$/m^3$','$g/m^3$',
        '$g/m^3$','$g/m^3$','$g/m^3$',
        'days','days']
Names = ['Height','LAI','RootWt','StemWt','LeafWt']
Pos = 1
for Var in Variables:
    NaNFilter = np.isnan(PredictedObserved.loc[:,'Predicted.'+Var]) | np.isnan(PredictedObserved.loc[:,'Observed.'+Var])
    IaNFilter = ~NaNFilter
    Obs = PredictedObserved.loc[:,'Observed.'+Var].loc[IaNFilter].values
    Pred = PredictedObserved.loc[:,'Predicted.'+Var].loc[IaNFilter].values
    residual = Pred - Obs
    DSWS = PredictedObserved.loc[:,'DReg'].loc[IaNFilter].values
    ax = Graph.add_subplot(3,3,Pos)
    plt.plot(DSWS,residual,Markers[MarPos],color = Colors[ColPos],label=Fol)
    uplim = max(residual.max(),Pred.max())*1.1
    lowlim = min(residual.min(),Pred.min())*0.95
    plt.title(Names[Pos-1] + ' (' + Units[Pos-1] + ')')
    plt.ylim(lowlim,uplim)
    plt.xlim(0,150)
    RegStats = MUte.MathUtilities.CalcRegressionStats(Var,Pred,Obs)
    LabelText = MakeLabel(RegStats)
    plt.text(0.05,0.78,LabelText,transform=ax.transAxes)
    Pos +=1
plt.text(70,-50,'Days since Cutting',fontsize=20)
plt.text(-40,30,'Residual',fontsize=20,rotation=90)
plt.legend(loc = (1.2,0),ncol=1,numpoints=1,markerscale=2)